# Qualità ciclabili

In [ ]:
import geopandas as gpd

from my_paths import *

In [ ]:
CRS_GRAD = "EPSG:4326"  # Lat-Long
CRS_METR = "EPSG:32632" # Metri

gdf_fontane = gpd.read_file(PATH_FONTANE_CLEAN)
gdf_ciclabili = gpd.read_file(PATH_CICLABILI_CLEAN)
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)

gdf_ciclabili = gdf_ciclabili.to_crs(CRS_METR)
gdf_fontane = gdf_fontane.to_crs(CRS_METR)
gdf_parchi = gdf_parchi.to_crs(CRS_METR)

gdf_fontane_con_buffer = gdf_fontane.buffer(500)
gdf_ciclabili = gdf_ciclabili.union_all()
gdf_parchi = gdf_parchi.union_all()

# Lontane 200m da ciclabili
fontanelle_lontane = gdf_fontane[~gdf_fontane_con_buffer.intersects(gdf_ciclabili)]
print(len(fontanelle_lontane))

# Non in dei parchi
fontanelle_lontane = fontanelle_lontane[~gdf_fontane_con_buffer.intersects(gdf_parchi)]
print(len(fontanelle_lontane))


In [133]:
fontanelle_lontane.to_crs(CRS_GRAD).to_file("./fontanelle_lontane.geojson", driver="GeoJSON")

# Sport e tempo libero

In [144]:
import requests
import json
from shapely import LineString, MultiLineString, Point
from shapely.ops import nearest_points

import geopandas as gpd
import polyline

from my_paths import *

In [145]:
PATH_ANALISI_TEST_CLEAN = "../Data/Clean/Analisi/test.geojson"

Carico le api key da file

In [146]:
PATH_API_KEY = "../Google_api_credentials/api_key_maps.json"
with open(PATH_API_KEY) as file: 
    API_KEY_GOOGLE = json.load(file)["key"]

In [171]:
gdf_ciclabili = gpd.read_file(PATH_CICLABILI_CLEAN)
gdf_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN)
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)
gdf_impianti_sportivi = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN)
gdf_fontane = gpd.read_file(PATH_FONTANE_CLEAN)
gdf_inquinamento = gpd.read_file(PATH_INQUINAMENTO_INGESTION_CLEAN)

Per eseguire operazioni di misurazione di distanze è meglio utilizzare crs metrico. Visto che i dataframe sono salvati con crs di tipo angolare (lat e long), li dobbiamo prima convertire.

In [148]:
CRS_GRAD = "EPSG:4326"  # Lat-Long
CRS_METR = "EPSG:32632" # Metri

## Parchi Isolati

In [149]:
gdf_ciclabili = gdf_ciclabili.to_crs(CRS_METR)
gdf_parchi = gdf_parchi.to_crs(CRS_METR)

Creiamo un "buffer" di contorno alle Multylinestring delle piste ciclabili (gli diamo spessore). Si è deciso per ora di  
impostare 200 --> 200 metri. Da verificare se modificare.  
Il buffer verrà usato per trovare intersezioni con le geometrie dei parchi.  
Il nuovo GeoDataFrame avrà tutti i parchi che NON si intersecano con le piste ciclabili.

In [150]:
gdf_ciclabili_buffer = gdf_ciclabili.buffer(200).union_all()
parchi_non_vicini = gdf_parchi[~gdf_parchi.geometry.intersects(gdf_ciclabili_buffer)]
print(len(parchi_non_vicini))

14


Trovati i parchi da "servire" con nuove ipotetiche piste ciclabili, andiamo a cercare i 2 punti più vicini tra la geometria delle piste ciclabili e i parchi. Scrorriamo quindi il dataframe dei parchi e facciamo la "nearest_points" delle due geometrie.

In [151]:
# Unisco tutte le piste ciclabili in un unica sola.  
gdf_ciclabili_unite = gpd.GeoDataFrame([gdf_ciclabili.union_all()], columns=["geometry"], crs=CRS_METR)

# Lista per raccogliere risultati
risultati = []

for idx_parco, parco in parchi_non_vicini.iterrows():
    # Trova i due punti più vicini tra geometria del parco e del tratto
    punto_parco, punto_ciclabile = nearest_points(parco["geometry"], gdf_ciclabili_unite["geometry"][0])
    percorso = LineString([punto_parco, punto_ciclabile])

    # Salva risultato
    risultati.append({
        'id_parco': parco["PARCO"],
        'distanza': percorso.length,
        'geometry': percorso
    })

# Crea GeoDataFrame del risultato
df_risultati_parchi = gpd.GeoDataFrame(risultati, geometry='geometry', crs=CRS_METR)

Dichiaro la funzione che utilizzeremo per trovare il percorso in bici tra i due punti grazie alle API di google maps.

In [152]:
def richiedi_percorso_google(origin:tuple[float, float], dest:tuple[float, float], api_key:str, mode:str = "walking"):
    """
    Restituisce una Multylinestring del percorso da punto A (origin) a punto B (dest).

    Args:
        origin (tuple[float, float]): Coordinate punto di partenza in formato: [Latitudine, Longitudine].
        dest (tuple[float, float]): Coordinate punto di arrivo in formato: [Latitudine, Longitudine].
        api_key (str): API_KEY di google maps per effettuare la richiesta.
        mode (str): Mezzo di trasporto da utilizzare per il calcolo: "driving", "walking", "bicycling", "transit".
                    "walking" verrà utilizzato di deafult.
    Returns:
        Multylinestring: Del tipo: [ [Longitudine, Latitudine], [Longitudine, Latitudine] ]. N.B: Sono inverse rispetto
                                    al formato dato in input, già pronte per kepler (google le usa al contrario).
    """

    outputFormat = "json"   # Usiamo sempre questo
    origin_str = f"{origin[1]},{origin[0]}"
    dest_str = f"{dest[1]},{dest[0]}"
    parameters = f"origin={origin_str}&destination={dest_str}&key={api_key}"
    url_API_directions = f"https://maps.googleapis.com/maps/api/directions/{outputFormat}?{parameters}&mode={mode}"

    risposta_json = requests.get(url_API_directions).json()
    coordinates = polyline.decode(risposta_json['routes'][0]['overview_polyline']['points'])
    coordinates = [[x[1], x[0]] for x in coordinates]
    return LineString(coordinates)

Trovo quindi le nuove geometrie dei percorsi di google maps e li vado a sostituire alla sua vecchia geometria che era riferita ai "nearest_points"

In [153]:
df_risultati_parchi = df_risultati_parchi.to_crs(CRS_GRAD)    # Google accetta lat-long
df_risultati_parchi["geometry"] = df_risultati_parchi["geometry"].apply(
    lambda x: richiedi_percorso_google(origin=x.coords[0],
                                      dest=x.coords[1],
                                      api_key=API_KEY_GOOGLE,
                                      mode="walking"))

Facciamo una eliminazione delle sovrapposizioni delle "nuove ciclabili" con quelle già esistenti per cercare di avere solo le nuove effettive piste da costruire.  
Per farlo bisogna lavorare di nuovo con il sistema "EPSG:32632" (metrico) e ci aiutiamo con un piccolo buffer (10 metri) sulle ciclabili.  
Per farlo si usa la funzione .difference delle Multilinestring.

In [154]:
df_risultati_parchi = df_risultati_parchi.to_crs(CRS_METR)    # Confronto geometrico usiamo metri

gpf_ciclabili_uniti_con_buffer = gdf_ciclabili.buffer(10).union_all()

df_risultati_parchi['geometry'] = df_risultati_parchi["geometry"].apply(
    lambda geom: geom.difference(gpf_ciclabili_uniti_con_buffer))

gdf_parchi_uniti = gdf_parchi.buffer(0.1).union_all()   # buffer minimo dovrebbe aiutare la precisione della difference
df_risultati_parchi['geometry'] = df_risultati_parchi["geometry"].apply(
    lambda geom: geom.difference(gdf_parchi_uniti))

# Aggiorno anche la lunghezza della ciclabile
df_risultati_parchi['lunghezza'] = df_risultati_parchi['geometry'].apply(lambda x: x.length)

## Impianti sportivi

In [176]:
gdf_ciclabili_buffer = gdf_ciclabili.to_crs(CRS_METR).buffer(200).union_all()
impianti_lontani = gdf_impianti_sportivi[~gdf_impianti_sportivi.to_crs(CRS_METR).geometry.intersects(gdf_ciclabili_buffer)]
impianti_lontani = impianti_lontani[~impianti_lontani.geometry.intersects(gdf_parchi.union_all())]

print(len(impianti_lontani))

72


In [177]:
impianti_lontani.to_file("../Data/Clean/Analisi/Impianti_lontani.geojson", driver="GeoJSON")

# Google API Monitor

Da usare ogni tanto per monitorare le chiamate api consumate.  
Difficilmente supereremo le 10k chiamate però non si sa mai.

N.B: Anche questa operazione ha un numero di chiamate limitate anche se non ho capito qual é ma facendola ogni tanto non mi preoccupo.

In [ ]:
from google.cloud import monitoring_v3
from datetime import datetime, timezone

In [ ]:
PATH_SERVICE_ACCOUNT = "../Google_api_credentials/service_account.json"

In [ ]:
client = monitoring_v3.MetricServiceClient.from_service_account_file(PATH_SERVICE_ACCOUNT)
project_id = "progetto-bike-aida"
project_name = f"projects/{project_id}"

start = datetime(2025, 7, 27)   # Data inizio utilizzo delle api
end = datetime.now(timezone.utc)

interval = monitoring_v3.TimeInterval({
    "start_time": {"seconds": int(start.timestamp())},
    "end_time": {"seconds": int(end.timestamp())},
})

results = client.list_time_series(
    request={
        "name": project_name,
        "filter": 'metric.type = "maps.googleapis.com/service/v2/request_count"',
        "interval": interval,
        "aggregation": {
            "alignment_period": {"seconds": 86400},
            "per_series_aligner": monitoring_v3.Aggregation.Aligner.ALIGN_SUM,
        },
        "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
    }
)

total = 0
for time_series in results:
    for point in time_series.points:
        total += point.value.int64_value

print(f"Totale chiamate Maps API: {total}")

Totale chiamate Maps API: 218
